In [1]:
import os
import sys

import os
cwd = os.getcwd()

cwd = '/'.join(cwd.split('/')[:-1])

def add_to_path(str):
    try:
        sys.path.index(str)
    except:
        sys.path.append(str)
        print(str + "\t add to the path list")
    else:
        print(str + "\t is already in the path list")

# this add all the libraris and scripts made for this project

add_to_path( cwd + "/datalib/")
# add_to_path( cwd + "/khinsider-master/")

/home/jahirmedinacs/repos/ts-fore_ESSALUD-2017-Central/piramidSong/datalib/	 add to the path list


In [2]:
import wave
import numpy as np
import scipy as sp

from matplotlib import pyplot as plt

import pprint
import struct

import pandas as pd

import time

# Part 1 : Information Retrive

## * Getting Sound Samples
In this case, despite the main idea was getting already noici samples, it was need change this, because the "master" sample needs to be almost pure , whit the minimal noise posible

In [3]:
# Download
# os.system("python3.6 ../khinsider-master/khinsider.py half-lifel-2-official-soundtrack")

In [4]:
DATA_PATH = "./"
files_wav = [ file for file in os.listdir(DATA_PATH) if file[-4:] == ".wav"]

In [5]:
files_wav

['keka_kalor.wav', 'pyramid_song.wav']

In [6]:
index = 0
sound_data_container = {}
for wav_f in files_wav:
    sound_data_container[index] = [wave.open(DATA_PATH + wav_f), wav_f]
    print(index , wav_f, "\n")
    index += 1

print(sound_data_container.__sizeof__())

0 keka_kalor.wav 

1 pyramid_song.wav 

264


In [7]:
def stereo_chanel_data(wav_obj_pointer, verbose=False):
    wav_obj_pointer.rewind()
    
    signed_data_l = []
    signed_data_r = []

    total_frames = wav_obj_pointer.getnframes()
    amount = 0

    for frame_index in range(1 , total_frames - 1, 1):
        wav_obj_pointer.setpos(frame_index)

        try:
            l_int = struct.unpack("<h",wav_obj_pointer.readframes(1)[:2])[0]
            r_int = struct.unpack("<h",wav_obj_pointer.readframes(1)[-2:])[0]

        except:
            if verbose:
                print("error with frame \t" , frame_index , " N_CHANEL_DATA ERROR \n")
        else:
            signed_data_l.append(l_int)
            signed_data_r.append(r_int)

            if frame_index % (44100*20) == 0 and verbose:
                print(round((frame_index / total_frames) * 100, 4), "\t%")
                print(l_int, "\t", r_int)
                print(len(signed_data_l), "\t", len(signed_data_r))

    return [signed_data_l, signed_data_r]

In [8]:
# test = sound_data_container[1][0]
# print(test.getparams())

In [9]:
def sound_csv(files, basic_verbose = False, verbose=False):
    
    for index in range(len(files)):
        
        if verbose:
            print(files[index][1], "\t in execution at : \t 0 % and going")
        elif basic_verbose:
            print(files[index][1], "\t in execution with index\t", index)
        else:
            pass
            
        [sound_obj_ref , sound_name] = files[index]
        
        data_l_r = stereo_chanel_data(sound_obj_ref, verbose)
        
        carry_data = np.matrix(data_l_r).T
        
        sound_frame_data = pd.DataFrame(carry_data, columns=["left", "rigth"])
                
        sound_frame_data.to_csv( (DATA_PATH + sound_name[:-4] + ".csv"), sep='\t', encoding='utf-8')
        
        if verbose or basic_verbose:
            print("\n", sound_name , "\tDONE\n")

In [10]:
start = time.time()
sound_csv(sound_data_container, basic_verbose=True)
print (time.time() - start)

keka_kalor.wav 	 in execution with index	 0

 keka_kalor.wav 	DONE

pyramid_song.wav 	 in execution with index	 1

 pyramid_song.wav 	DONE

315.808256149292


In [11]:
def meta_data(files, verbose=False):
    specs_matrix = []
    
    for index in range(len(files)):
        if verbose:
            print(files[index][1], "\t in transcription")
        else:
            pass
        
        [sound_obj_ref , sound_name] = files[index]
        
        specs_matrix.append([sound_name] + list(sound_obj_ref.getparams()))
        
        if verbose:
            print("\n", sound_name , "\tDONE\n")
    
    meta_sound_data = pd.DataFrame(specs_matrix, columns=
                                   ["file name", "nchannels", "sampwidth", "framerate", "nframes", "comptype", "compname"])
                
    meta_sound_data.to_csv( (DATA_PATH + "meta_sound_data.csv"), sep='\t', encoding='utf-8')
    
    

In [12]:
meta_data(sound_data_container, True)

keka_kalor.wav 	 in transcription

 keka_kalor.wav 	DONE

pyramid_song.wav 	 in transcription

 pyramid_song.wav 	DONE

